In [ ]:
'''
Avoid turning off your notebook or timeout sessions using this:
 
Mozilla:
  type:
  >ctrl + shift + i
 
  Go to console 
 
  Paste and run this:
  >>
  function ClickConnect(){
  console.log("Working"); 
  document.querySelector("colab-toolbar-button#connect").click() 
  }setInterval(ClickConnect,60000)
'''

'\nAvoid turning off your notebook or timeout sessions using this:\n \nMozilla:\n  type:\n  >ctrl + shift + i\n \n  Go to console \n \n  Paste and run this:\n  >>\n  function ClickConnect(){\n  console.log("Working"); \n  document.querySelector("colab-toolbar-button#connect").click() \n  }setInterval(ClickConnect,60000)\n'

In [ ]:
# Save points you sampled to your drive
path_save_samples = '/drive/My Drive/mhar_paper_samples_2/'
# Save samples times to your drive
path_save_times = '/drive/My Drive/mhar_test_times_2/'
path_save_times_p100 = '/drive/My Drive/mhar_test_times_p100/'
 
# Run times?
run_times = False

In [ ]:
!nvidia-smi

Mon Jul  6 08:15:30 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Mount google drive
gpu_ = 'p100'
from google.colab import drive
drive.mount('/drive') 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /drive


In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2200.000
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	: 4400.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 b

In [ ]:
!cat /proc/meminfo

MemTotal:       13333556 kB
MemFree:        10439648 kB
MemAvailable:   12439744 kB
Buffers:           76124 kB
Cached:          2079720 kB
SwapCached:            0 kB
Active:           765088 kB
Inactive:        1841756 kB
Active(anon):     429260 kB
Inactive(anon):      332 kB
Active(file):     335828 kB
Inactive(file):  1841424 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:              7040 kB
Writeback:            16 kB
AnonPages:        450944 kB
Mapped:           245908 kB
Shmem:               948 kB
Slab:             167316 kB
SReclaimable:     127468 kB
SUnreclaim:        39848 kB
KernelStack:        4176 kB
PageTables:         5908 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6666776 kB
Committed_AS:    2801860 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
Percpu:              920 kB
AnonHugePages:   

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [ ]:
!python --version

Python 3.6.9


In [ ]:
# pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
import torch
import numpy as np
import pandas as pd

In [ ]:
# Set float 64 as default precision, can be changed to 32
torch.set_default_dtype(torch.float64)

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [ ]:
'''
Auxiliary routines for the mhar, projection, random numbers, etc
'''

# Create Projection Matrix
def create_projection_matrix(ae, check=True, device='cpu'):
  '''
  ae : tensor with the equality matrix, must be a matrix
  check: Boolean for evaluating numerical estability of the inversion
  '''
  aet = torch.transpose(ae,0,1)
  ae_aux = torch.matmul(ae, aet)
  ae_inv = torch.inverse(ae_aux)
  # Check numerycal estability
  if check:
    est = torch.mm(ae_inv, ae_aux)
    est = torch.max(torch.abs(est - torch.eye(est.shape[0], device=device)))
    print('Max non zero error: ', est)
    del est

  la = torch.matmul(aet, ae_inv)
  projection_matrix = torch.matmul(la,ae)
  projection_matrix = torch.eye(projection_matrix.shape[0], device=device) - projection_matrix
  

  # Free Memory
  del ae_aux
  del ae_inv
  del la

  return projection_matrix



# Draw random numbers
def create_h(n,z, generator,device='cpu'):

  if '64' in str(torch.get_default_dtype()):
    if device != 'cpu':
      h = torch.cuda.DoubleTensor(z,n,1).normal_(generator=generator)
    else:
      h = torch.DoubleTensor(z, n,1).normal_(generator=generator)
  else:
    if device != 'cpu':
      h = torch.cuda.DoubleTensor(z,n,1).normal_(generator=generator)
    else:
      h = torch.DoubleTensor(z,n,1).normal_(generator=generator)
  return h

 # Draw random numbers
def draw_uniform(z, generator,device='cpu'):

  # if 64 
  if '64' in str(torch.get_default_dtype()):
    if device != 'cpu':
      h = torch.cuda.DoubleTensor(z,1).uniform_(generator=generator)
    else:
      h = torch.DoubleTensor(z,1).uniform_(generator=generator)
  else:
    if device != 'cpu':
      h = torch.cuda.FloatTensor(z,1).uniform_(generator=generator)
    else:
      h = torch.FloatTensor(z,1).uniform_(generator=generator)
  return h 




In [ ]:
# MHAR
def mhar(z, 
         ai=torch.empty(0), bi=torch.empty(0), 
         ae=torch.empty(0), be=torch.empty(0), 
         x_0=torch.empty(0), 
         T=1,seed=None, thinning = None,
         check=True, device='cpu', save=True):
  
  print(device)

  ##### Set min and max values
  min_ = torch.finfo(torch.get_default_dtype()).min + 2.0
  max_ = torch.finfo(torch.get_default_dtype()).max - 2.0
  eps_ = torch.finfo(torch.get_default_dtype()).eps
  tiny_ = torch.finfo(torch.get_default_dtype()).tiny

  #### Set seed
  random_gen = torch.Generator(device=device)
  if seed:
    random_gen.manual_seed(seed)
  else:
    random_gen.seed()

  #### Get dimensions
  n = ai.shape[1]
  mi = ai.shape[0]
  me = ae.shape[0]
  print('n: ', n, '  mi:', mi, '  me:', me, '  z:', z)
  # Check if the polytope is full dimensional
  non_full = me > 0

  #### Set thinning factor
  if thinning:
    pass
  else:
    thinning = int((n-1)*(n-1)*(n-1))
  print('Thinning factor:', thinning)



  #### Copy tensors to gpu
  if 'cpu' not in str(device):
    ai = ai.cuda()
    bi = bi.cuda()
    x = x_0.cuda()
    if non_full:
      ae = ae.cuda()
      be = be.cuda()
  else:
    x = x_0

  ####Create projection matrix
  if non_full:
    pr = create_projection_matrix(ae, check, device)

  #### Pad Matrix
  x = x.repeat(z, 1, 1)
  

  #################### Iteration Loop #######################
  t = 1
  burned = 0

  while t<=T:
    torch.cuda.empty_cache()

    #### Draw direction
    h = create_h(n,z, generator=random_gen,device=device)

    #### Project
    if non_full:
      d = torch.matmul(pr,h)
    else:
      d = h
    ##### B - AX / AD
    numerator = bi - torch.matmul(ai,x)
    denominator = torch.matmul(ai,d)
    # Overwrite nuemrator to keep memory free
    numerator = numerator/denominator
    # From the positive denominators you want the smallest one
    lambda_pos = torch.min(~(denominator > 0.0)*max_ + 
                           (denominator > 0.0)*numerator,1).values
    # From the negative denominators you want the biggest
    lambda_neg = torch.max(~(denominator < 0.0)*min_ + 
                           (denominator < 0.0)*numerator,1).values
    #### Uniform draw
    u = draw_uniform(z, generator=random_gen,device=device)
    theta = (1.0- u)*lambda_pos + (u)*lambda_neg 
    
    ##### New X
    x = x + d*theta[:,None]

    ##### Manage the burning rate and save points
    if burned >= thinning:
      print(t)

      if (t == 1) & (save):
        X = torch.flatten(x, start_dim=1)
        X = X.cpu()
      if (t > 1) & (save):
        X = torch.cat((X, 
                       torch.flatten(x, start_dim=1).cpu()), 
                      dim=0)
      t = t + 1
      burned = -1

    burned = burned + 1

  if save:
    return X
  else:
    return 0









In [ ]:
'''
Hypercube Creation
'''
import time

def create_hyper(n):
  # Create Inequalities
  hyper_ai = torch.cat((torch.eye(n),torch.eye(n)*-1.0), dim=0)
  hyper_bi = torch.from_numpy(np.array([1]*2*n))[:,None]

  return hyper_ai, hyper_bi

def create_inner_point_hyper(n):
  x_0 = torch.empty(n, 1, dtype=torch.float64)
  x_0.fill_(0.01)
  return x_0

def debug_hyper(n, z=1, thinning=100, device='cpu', save=True, T=1, seed=123):
 
  hyper_ai, hyper_bi = create_hyper(n)
  x_0 = create_inner_point_hyper(n)

  start_time = time.time()
  X = mhar(z = z,
       ai= hyper_ai, 
       bi = hyper_bi,
       x_0 = x_0, 
       seed=seed, 
       check=True,
       T = T,
       thinning=thinning,
       device=device,
       save=save)
  time_=  (time.time() - start_time)

  print('End')
  return X, time_

In [ ]:
'''
Simplex Creation
'''
def create_simplex(n):
  # Create Inequalities
  simplex_ai = torch.eye(n)*-1.0
  simplex_bi = torch.empty(n, 1, dtype=torch.float64)
  simplex_bi.fill_(0.0)

  # Create Equalities
  simplex_ae = torch.empty(1, n, dtype=torch.float64)
  simplex_ae.fill_(1.0)
  simplex_be = torch.empty(1, 1, dtype=torch.float64)
  simplex_be.fill_(1.0)
  return simplex_ai, simplex_bi, simplex_ae, simplex_be

def create_inner_point_simplex(n):
  x_0 = torch.empty(n, 1, dtype=torch.float64)
  x_0.fill_(1.0/n)
  return x_0

def debug_simplex(n, z=1,thinning=100,device='cpu', save=True, T=1, seed=123):
 
  simplex_ai, simplex_bi, simplex_ae, simplex_be = create_simplex(n)
  x_0 = create_inner_point_simplex(n)

  start_time = time.time()
  X = mhar(z = z,
       ai= simplex_ai, 
       bi = simplex_bi,
       ae = simplex_ae, 
       be = simplex_be, 
       x_0 = x_0, 
       seed=seed, 
       check=True,
       T = T,
       thinning=thinning,
       device=device,
       save=save)
  print('End')
  time_=  (time.time() - start_time)
  return X, time_

In [ ]:

# X, t = debug_hyper(n=10, z=100, thinning=100000, device='cuda', 
#                    T=1, save=True)

# X = debug_simplex(n=1000, z=100, thinning=100000, device='cuda', T=1, save=False)
# torch.sum(X, dim=1)


In [ ]:
import time
import pandas as pd
from random import randint


def run_time_sims(n_list = [],
                  z_list = [],
                  thinning_ = 30000,
                  figure = 'simplex',
                  save_pp = '/drive/My Drive/mhar_test_times_2/',
                  postfix = '_data.csv'):
  
  drive.mount('/drive') 
  l_time_ = []
  l_z_ = []
  l_n_ = []

  save_hash= randint(1,1000000000000)

  for n_ in n_list:
      for z_ in z_list:
        start_time = time.time()

        if figure == 'simplex':
          X, time_ = debug_simplex(n=n_, z=z_,thinning=thinning_,device='cuda', 
                      save=False)
        elif 'hypercube' == figure:
          X, time_ = debug_hyper(n=n_, z=z_,thinning=thinning_,device='cuda',
                      save=False)
        
        points_ = z_*thinning_
        l_time_.append(time_)
        l_z_.append(z_)
        l_n_.append(n_)

        print("--- %s seconds ---" % (time_))
        print("--- %s points per sec ---" % (points_/time_))
        performance = pd.DataFrame.from_dict({'n':l_n_, 'z':l_z_, 
                                              'time': l_time_})
        performance['thi'] = thinning_ 
        performance['gpu'] = gpu_ 
        performance['figure'] = figure

        performance['samples_sec'] = performance['z'] * thinning_ / performance['time']
        name = save_pp + str(save_hash) + postfix
        print(name)
        performance.to_csv(name, index=False)



In [ ]:
if run_times:    
  # Low dimensional Simplex

  for it in range(1,11):
    run_time_sims(n_list = [3, 5, 15, 25, 50],
                      z_list = [1, 500, 1000, 2500, 5000, 10000],
                      thinning_ = 30000,
                      figure = 'simplex',
                      save_pp = path_save_times,
                      postfix = '_data.csv')
    print()
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print(it)
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print()


  # Medium dimensional Simplex

  for it in range(1,11):
    run_time_sims(n_list = [100, 250, 300, 500, 1000],
                      z_list = [1, 500, 1500, 3000, 4000],
                      thinning_ = 30000,
                      figure = 'simplex',
                      save_pp = path_save_times,
                      postfix = '_data.csv')
    print()
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print(it)
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print()

  # HIgh dimensional Simplex

  for it in range(1,11):
    run_time_sims(n_list = [2500, 5000],
                      z_list = [500, 1000],
                      thinning_ = 10000,
                      figure = 'simplex',
                      save_pp = path_save_times,
                      postfix = '_data.csv')
    print()
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print(it)
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print()



In [ ]:
if run_times:  
  # Low dimensional hypercube

  for it in range(1,11):
    run_time_sims(n_list = [3, 5, 15, 25, 50],
                      z_list = [1, 500, 1000, 2500, 5000, 10000],
                      thinning_ = 30000,
                      figure = 'hypercube',
                      save_pp = path_save_times,
                      postfix = '_data.csv')
    print()
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print(it)
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print()

    # Medium dimensional hypercube

  for it in range(1,11):
    run_time_sims(n_list = [100, 250, 500, 1000],
                      z_list = [1, 500, 1500, 3000, 4000],
                      thinning_ = 30000,
                      figure = 'hypercube',
                      save_pp = path_save_times,
                      postfix = '_data.csv')
    print()
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print(it)
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print()

  # HIgh dimensional hypercube

  for it in range(1,11):
    run_time_sims(n_list = [2500, 5000],
                      z_list = [500, 1000, 1500],
                      thinning_ = 10000,
                      figure = 'hypercube',
                      save_pp = path_save_times,
                      postfix = '_data.csv')
    print()
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print(it)
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print()


In [ ]:
import time
import pandas as pd
from random import randint
from os import listdir
from os.path import isfile, join

def run_point_sims(n_list = [],
                  z_list = [],
                  thinning_ = None,
                  figure = 'simplex',
                  save_pp = '/drive/My Drive/mhar_paper_samples/',
                  T = 1,
                   seed = 1):
  
  drive.mount('/drive') 
  l_time_ = []
  l_z_ = []
  l_n_ = []
    
  for n_ in n_list:
    for z_ in z_list:

      name = str(seed) + '_' + figure + '%' + str(z_) + '%' + str(n_) + '.csv'
      # Check if the simulation already exists
      onlyfiles = [f for f in listdir(save_pp) if isfile(join(save_pp, f))]
      has_to_run = name not in onlyfiles

      if has_to_run:
        if figure == 'simplex':
          if not None:
            thinning_ = (n_ - 1)*(n_ - 1)*(n_ - 1)
          X, time_ = debug_simplex(n=n_, z=z_,thinning=thinning_,device='cuda', 
                        save=True, seed=seed, T=T)
        elif 'hypercube' == figure:
          if not None:
            thinning_ = (n_)*(n_)*(n_)         
          X, time_ = debug_hyper(n=n_, z=z_,thinning=thinning_,device='cuda',
                        save=True, seed=seed, T=T)
          
        path_save = save_pp + name # Save path
        print(path_save)
        X_data = pd.DataFrame(X.numpy())
        X_data.to_csv(path_save, index=False)
        print(name)
      else:
        print(name, ' existed already')



In [ ]:
# for it in range(1,11):
for it in range(1,11):
  run_point_sims(n_list = [3, 15, 25, 50],
                    z_list = [1000],
                    thinning_ = None,
                    figure = 'hypercube',
                    save_pp = path_save_samples,
                    T = 10,
                    seed = it * 133
                )
  run_point_sims(n_list = [3, 15, 25, 50],
                    z_list = [1000],
                    thinning_ = None,
                    figure = 'simplex',
                    save_pp = path_save_samples,
                    T = 10,
                    seed = it * 133
                )
  print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
  print(it)
  print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).
133_hypercube%1000%3.csv  existed already
133_hypercube%1000%15.csv  existed already
133_hypercube%1000%25.csv  existed already
133_hypercube%1000%50.csv  existed already
Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).
133_simplex%1000%3.csv  existed already
133_simplex%1000%15.csv  existed already
133_simplex%1000%25.csv  existed already
133_simplex%1000%50.csv  existed already
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
1
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).
266_hypercube%1000%3.csv  existed already
266_hypercube%1000%15.csv  existed already
266_hypercube%1000%25.csv  existed already
266_hypercube%1000%50.csv  existed already
Drive already mounted at /drive;